# TCRBert: Improves Prediction of Epitope-specific TCR recognition using Pre-trained Protein Embeddings.

## Global configurations

In [1]:
import logging
import logging.config
import os
import sys
import warnings
from enum import auto
import pandas as pd

rootdir = '/home/hym/trunk/TCRBert'
workdir = '%s/notebook' % rootdir
datadir = '%s/data' % rootdir
srcdir = '%s/tcrbert' % rootdir
outdir = '%s/output' % rootdir

os.chdir(workdir)

sys.path.append(rootdir)
sys.path.append(srcdir)

# Display
pd.set_option('display.max.rows', 999)
pd.set_option('display.max.columns', 999)

# Logger
warnings.filterwarnings('ignore')
logging.config.fileConfig('../config/logging.conf')
logger = logging.getLogger('tcrbert')

sys.path

['/home/hym/trunk/TCRBert/notebook',
 '/home/hym/trunk/TCRBert/notebook',
 '/home/hym/trunk/epidab',
 '/home/hym/trunk/epidab/epidab',
 '/home/hym/trunk/kvacc',
 '/home/hym/trunk/kvacc/kvacc',
 '/home/hym/trunk/lib',
 '/home/hym/trunk',
 '/home/hym/anaconda3/envs/py37-torch/lib/python37.zip',
 '/home/hym/anaconda3/envs/py37-torch/lib/python3.7',
 '/home/hym/anaconda3/envs/py37-torch/lib/python3.7/lib-dynload',
 '',
 '/home/hym/anaconda3/envs/py37-torch/lib/python3.7/site-packages',
 '/home/hym/anaconda3/envs/py37-torch/lib/python3.7/site-packages/IPython/extensions',
 '/home/hym/.ipython',
 '/home/hym/trunk/TCRBert',
 '/home/hym/trunk/TCRBert/tcrbert']

## Training Datasets

In [2]:
from enum import auto
from tcrbert.commons import StrEnum

class TCREpitopeDFColumnName(StrEnum):
    epitope = auto()
    epitope_gene = auto()
    epitope_species = auto()
    species = auto()
    cdr3b = auto()
    mhc = auto()
    source = auto()
    label = auto()

    @classmethod
    def values(cls):
        return [c.value for c in cls]

def get_index(row, sep='_'):
    return '%s%s%s' % (row[CN.epitope], sep, row[CN.cdr3b])

CN = TCREpitopeDFColumnName


### Postitive dataset

#### Dash data
- containing epitope-specific paired epitope-specific paired TCRα and TCRβ chains for  three epitopes from humans and for seven epitopes from mice[{Dash:2017go}]

In [9]:
import pandas as pd
from IPython.core.display import display
from collections import OrderedDict

gene_info_map = OrderedDict({
    'BMLF': ('EBV', 'GLCTLVAML', 'HLA-A*02:01'),
    'pp65': ('CMV', 'NLVPMVATV', 'HLA-A*02:01'),
    'M1': ('IAV', 'GILGFVFTL', 'HLA-A*02:01'),
    'F2': ('IAV', 'LSLRNPILV', 'H2-Db'),
    'NP': ('IAV', 'ASNENMETM', 'H2-Db'),
    'PA': ('IAV', 'SSLENFRAYV', 'H2-Db'),
    'PB1': ('IAV', 'SSYRRPVGI', 'H2-Kb'),
    'm139': ('mCMV', 'TVYGFCLL', 'H2-Kb'),
    'M38': ('mCMV', 'SSPPMFRV', 'H2-Kb'),
    'M45': ('mCMV', 'HGIRNASFI', 'H2-Db'),   
})

fn_dash = '%s/Dash/human_mouse_pairseqs_v1_parsed_seqs_probs_mq20_clones.tsv' % datadir
df = pd.read_table(fn_dash, sep='\t')
df = df.dropna(subset=['epitope', 'cdr3b'])
df[CN.epitope_gene] = df['epitope']
df[CN.epitope_species] = df[CN.epitope_gene].map(lambda x: gene_info_map[x][0])
df[CN.epitope] = df[CN.epitope_gene].map(lambda x: gene_info_map[x][1])
df[CN.mhc] = df[CN.epitope_gene].map(lambda x: gene_info_map[x][2])
df[CN.species] = df['subject'].map(lambda x: 'human' if 'human' in x else 'mouse')
df[CN.cdr3b] = df['cdr3b'].str.strip().str.upper()
df[CN.source] = 'Dash'
df[CN.label] = 1
df = df.loc[:, CN.values()]
df.index = df.apply(lambda row: get_index(row), axis=1)

df = df[~df.index.duplicated()]
display(df.head(), df.shape)

for i, gene in enumerate(gene_info_map.keys()):
    subdf = df[df[CN.epitope_gene] == gene]
    epitope = subdf[CN.epitope].iat[0]
    species = subdf[CN.epitope_species].iat[0]
    n_cdr3b = subdf.shape[0]
    print('%s: Epitope: %s, species: %s, gene: %s, n_cdr3b: %s' % (i + 1, epitope, species, gene, n_cdr3b))
    
df_dash = df

,epitope,epitope_gene,epitope_species,species,cdr3b,mhc,source,label
GLCTLVAML_CASSLWTGSHEQYF,GLCTLVAML,BMLF,EBV,human,CASSLWTGSHEQYF,HLA-A*02:01,Dash,1
GLCTLVAML_CASSQSPGGEQYF,GLCTLVAML,BMLF,EBV,human,CASSQSPGGEQYF,HLA-A*02:01,Dash,1
GLCTLVAML_CASSLTTEQQFF,GLCTLVAML,BMLF,EBV,human,CASSLTTEQQFF,HLA-A*02:01,Dash,1
GLCTLVAML_CSARDATGNGYTF,GLCTLVAML,BMLF,EBV,human,CSARDATGNGYTF,HLA-A*02:01,Dash,1
GLCTLVAML_CSARDSTGNGYTF,GLCTLVAML,BMLF,EBV,human,CSARDSTGNGYTF,HLA-A*02:01,Dash,1


(1408, 8)

1: Epitope: GLCTLVAML, species: EBV, gene: BMLF, n_cdr3b: 54
2: Epitope: NLVPMVATV, species: CMV, gene: pp65, n_cdr3b: 54
3: Epitope: GILGFVFTL, species: IAV, gene: M1, n_cdr3b: 150
4: Epitope: LSLRNPILV, species: IAV, gene: F2, n_cdr3b: 102
5: Epitope: ASNENMETM, species: IAV, gene: NP, n_cdr3b: 149
6: Epitope: SSLENFRAYV, species: IAV, gene: PA, n_cdr3b: 230
7: Epitope: SSYRRPVGI, species: IAV, gene: PB1, n_cdr3b: 335
8: Epitope: TVYGFCLL, species: mCMV, gene: m139, n_cdr3b: 73
9: Epitope: SSPPMFRV, species: mCMV, gene: M38, n_cdr3b: 60
10: Epitope: HGIRNASFI, species: mCMV, gene: M45, n_cdr3b: 201


#### VDJdb

- We extend the positive dataset with new data set from VDJdb{Bagaev:2019hf}](https://vdjdb.cdr3.net, downloaded May 2021), which is a manually curated database that contains TCR sequences with known antigen specificity. Every entry in VDJdb has been given a confidence score between 0  and 3 (0: critical information missing, 1: medium confidence, 2: high confidence, 3: very high  confidence). 
- Confidence score가 죄소한 1이상인 entry를 선택하였다
- MHC-I restricted 에피토프들만 선택하였다.

In [10]:
fn_vdjdb = '%s/VDJdb/vdjdb_20210201.txt' % datadir
################################

df = pd.read_table(fn_vdjdb, sep='\t', header=0)
logger.debug('Current df.shape: %s' % str(df.shape))

# Select beta CDR3 sequence
logger.debug('Select beta CDR3 sequences and MHC-I restricted epitopes')
df = df[(df['gene'] == 'TRB') & (df['mhc.class'] == 'MHCI')]
logger.debug('Current df.shape: %s' % str(df.shape))

# Check valid CDR3 and peptide sequences
logger.debug('Select valid CDR3 and epitope sequences')
df = df.dropna(subset=['cdr3', 'antigen.epitope'])
logger.debug('Current df.shape: %s' % str(df.shape))

logger.debug('Select confidence score > 0')
df = df[df['vdjdb.score'].map(lambda score: score > 0)]
logger.debug('Current df.shape: %s' % str(df.shape))

df[CN.epitope] = df['antigen.epitope'].str.strip().str.upper()
df[CN.epitope_species] = df['antigen.species']
df[CN.epitope_gene] = df['antigen.gene']
df[CN.species] = df['species']
df[CN.cdr3b] = df['cdr3'].str.strip().str.upper()
# df[CN.mhc] = df['mhc.a'].map(lambda x: MHCAlleleName.sub_name(MHCAlleleName.std_name(x)))
df[CN.mhc] = df['mhc.a']
df[CN.source] = 'VDJdb'
df[CN.label] = 1

df.index = df.apply(lambda row: get_index(row), axis=1)
logger.debug('Drop duplicates with the same{epitope, CDR3b}')
df = df[~df.index.duplicated()]
logger.debug('Current df.shape: %s' % str(df.shape))

df = df.loc[:, CN.values()]

# logger.debug('Select all epitope with at least 30 TCRB sequences')
# tmp = df[CN.epitope].value_counts()
# tmp = tmp[tmp >= n_cdr3b_cutoff]
# df = df[df[CN.epitope].map(lambda x: x in tmp.index)]
# logger.debug('Final df.shape: %s' % str(df.shape))

display(df.head(), df.shape)

for i, (epitope, subdf) in enumerate(df.groupby([CN.epitope])):
    epitope_gene = subdf[CN.epitope_gene].unique()
    epitope_species = subdf[CN.epitope_species].iat[0]
    n_cdr3b = subdf.shape[0]

    print(
        '%s: Epitope: %s(%s), speices: %s, gene: %s, n_cdr3b: %s' %
        (i + 1, epitope, len(epitope), epitope_species, epitope_gene, n_cdr3b))

df_vdjdb = df

2021-06-01 03:54:06 [DEBUG]: Current df.shape: (78701, 21)
2021-06-01 03:54:06 [DEBUG]: Select beta CDR3 sequences and MHC-I restricted epitopes
2021-06-01 03:54:06 [DEBUG]: Current df.shape: (44797, 21)
2021-06-01 03:54:06 [DEBUG]: Select valid CDR3 and epitope sequences
2021-06-01 03:54:06 [DEBUG]: Current df.shape: (44797, 21)
2021-06-01 03:54:06 [DEBUG]: Select confidence score > 0
2021-06-01 03:54:06 [DEBUG]: Current df.shape: (6659, 21)
2021-06-01 03:54:06 [DEBUG]: Drop duplicates with the same{epitope, CDR3b}
2021-06-01 03:54:06 [DEBUG]: Current df.shape: (4275, 28)


,epitope,epitope_gene,epitope_species,species,cdr3b,mhc,source,label
FLKEKGGL_CASSYLPGQGDHYSNQPQHF,FLKEKGGL,Nef,HIV-1,HomoSapiens,CASSYLPGQGDHYSNQPQHF,HLA-B*08,VDJdb,1
FLKEKGGL_CASSFEAGQGFFSNQPQHF,FLKEKGGL,Nef,HIV-1,HomoSapiens,CASSFEAGQGFFSNQPQHF,HLA-B*08,VDJdb,1
FLKEKGGL_CASSFEPGQGFYSNQPQHF,FLKEKGGL,Nef,HIV-1,HomoSapiens,CASSFEPGQGFYSNQPQHF,HLA-B*08,VDJdb,1
FLKEKGGL_CASSYEPGQVSHYSNQPQHF,FLKEKGGL,Nef,HIV-1,HomoSapiens,CASSYEPGQVSHYSNQPQHF,HLA-B*08,VDJdb,1
FLKEKGGL_CASSALASLNEQFF,FLKEKGGL,Nef,HIV-1,HomoSapiens,CASSALASLNEQFF,HLA-B*08,VDJdb,1


(4275, 8)

1: Epitope: AAFKRSCLK(9), speices: MCPyV, gene: ['T-Ag'], n_cdr3b: 5
2: Epitope: AAGIGILTV(9), speices: HomoSapiens, gene: ['MLANA'], n_cdr3b: 2
3: Epitope: ALDPHSGHFV(10), speices: HomoSapiens, gene: ['CDK4'], n_cdr3b: 4
4: Epitope: ALGIGILTV(9), speices: HomoSapiens, gene: ['MLANA'], n_cdr3b: 1
5: Epitope: ALSPVIPHI(9), speices: HomoSapiens, gene: ['MLL2'], n_cdr3b: 2
6: Epitope: ALTPVVVTL(9), speices: HomoSapiens, gene: ['CDK4'], n_cdr3b: 6
7: Epitope: ALWGPDPAAA(10), speices: HomoSapiens, gene: ['INS'], n_cdr3b: 1
8: Epitope: ALYGFVPVL(9), speices: HomoSapiens, gene: ['GANAB'], n_cdr3b: 6
9: Epitope: APARLERRHSA(11), speices: HomoSapiens, gene: ['KIF16B'], n_cdr3b: 3
10: Epitope: APRGPHGGAASGL(13), speices: HomoSapiens, gene: ['NY-ESO-1' 'NY-ESO'], n_cdr3b: 5
11: Epitope: ARMILMTHF(9), speices: HCV, gene: ['NS5B'], n_cdr3b: 14
12: Epitope: ASNENMETM(9), speices: InfluenzaA, gene: ['NP'], n_cdr3b: 69
13: Epitope: ATDALMTGY(9), speices: HCV, gene: ['NS3'], n_cdr3b: 135
14: Epitope: A

#### McPAS-TCR

- McPAS-TCR(http://friedmanlab.weizmann.ac.il/McPAS-TCR/) is a manually curated catalog of pathology-associated TCR sequences[{Tickotsky:2017bo}].
- MHC-I restricted 에피토프들을 선택하였다.

In [5]:
!head -n 1 ../data/McPAS/McPAS-TCR_20210521.csv

"CDR3.alpha.aa","CDR3.beta.aa","Species","Category","Pathology","Pathology.Mesh.ID","Additional.study.details","Antigen.identification.method","Single.cell","NGS","Antigen.protein","Protein.ID","Epitope.peptide","Epitope.ID","MHC","Tissue","T.Cell.Type","T.cell.characteristics","CDR3.alpha.nt","TRAV","TRAJ","TRBV","TRBD","TRBJ","Reconstructed.J.annotation","CDR3.beta.nt","Mouse.strain","PubMed.ID","Remarks"


In [43]:
tokens

['YMLDLQPET', 'KLPQLCTEL']

In [45]:
import numpy as np

fn_mcpas = '%s/McPAS/McPAS-TCR_20210521.csv' % datadir
################################

df = pd.read_csv(fn_mcpas)
logger.debug('Current df.shape: %s' % str(df.shape))

# Select valid beta CDR3 sequence and epitope sequence
logger.debug('Select valid beta CDR3 and epitope sequences')
df = df.dropna(subset=['CDR3.beta.aa', 'Epitope.peptide'])
logger.debug('Current df.shape: %s' % str(df.shape))


# df[CN.epitope] = df['Epitope.peptide'].map(lambda x: x.split('/')[0].upper())
df[CN.epitope] = df['Epitope.peptide'].str.strip().str.upper()

# Handle multiple epitope
logger.debug('Extend by multi-epitopes')
tmpdf = df[df[CN.epitope].str.contains('/')].copy()
for mepitope, subdf in tmpdf.groupby([CN.epitope]):
    logger.debug('Multi epitope: %s' % mepitope)
    tokens = mepitope.split('/')
    logger.debug('Convert epitope: %s to %s' % (mepitope, tokens[0]))
    df[CN.epitope][df[CN.epitope] == mepitope] = tokens[0]    

    for epitope in tokens[1:]:
        logger.debug('Extend by epitope: %s' % epitope)
        subdf[CN.epitope] = epitope
        df = df.append(subdf)
logger.debug('Current df.shape: %s' % (str(df.shape)))
        
df[CN.epitope_gene] = None
df[CN.epitope_species] = df['Pathology']
df[CN.species] = df['Species']
df[CN.cdr3b] = df['CDR3.beta.aa'].str.strip().str.upper()
df[CN.mhc] = df['MHC'].str.strip()
df[CN.source] = 'McPAS'
df[CN.label] = 1

df.index = df.apply(lambda row: get_index(row), axis=1)

logger.debug('Select MHC-I restricted entries')
df = df[
    (df[CN.mhc].notnull()) &
    (np.logical_not(df[CN.mhc].str.contains('DR|DP|DQ')))
]
logger.debug('Current df.shape: %s' % str(df.shape))

logger.debug('Drop duplicates with the same{epitope, CDR3b}')
df = df[~df.index.duplicated()]
logger.debug('Current df.shape: %s' % str(df.shape))

df = df.loc[:, CN.values()]

# logger.debug('Select all epitope with at least 30 TCRB sequences')
# tmp = df[CN.epitope].value_counts()
# tmp = tmp[tmp >= n_cdr3b_cutoff]
# df = df[df[CN.epitope].map(lambda x: x in tmp.index)]
# logger.debug('Final df.shape: %s' % str(df.shape))

display(df.head(), df.shape)
for i, (epitope, subdf) in enumerate(df.groupby([CN.epitope])):
#     epitope_gene = ','.join(subdf[CN.epitope_gene].unique())
    epitope_species = subdf[CN.epitope_species].iat[0]
    n_cdr3b = subdf.shape[0]
    
    print('%s: Epitope: %s(%s), speices: %s, n_cdr3b: %s' % (i + 1, epitope, len(epitope), 
                                                             epitope_species, n_cdr3b))
    
df_mcpas = df

2021-06-01 04:49:26 [DEBUG]: Current df.shape: (39045, 29)
2021-06-01 04:49:26 [DEBUG]: Select valid beta CDR3 and epitope sequences
2021-06-01 04:49:26 [DEBUG]: Current df.shape: (15051, 29)
2021-06-01 04:49:26 [DEBUG]: Extend by multi-epitopes
2021-06-01 04:49:26 [DEBUG]: Multi epitope: YMLDLQPET/KLPQLCTEL
2021-06-01 04:49:26 [DEBUG]: Convert epitope: YMLDLQPET/KLPQLCTEL to YMLDLQPET
2021-06-01 04:49:26 [DEBUG]: Extend by epitope: KLPQLCTEL
2021-06-01 04:49:26 [DEBUG]: Current df.shape: (15096, 30)
2021-06-01 04:49:26 [DEBUG]: Select MHC-I restricted entries
2021-06-01 04:49:26 [DEBUG]: Current df.shape: (13597, 37)
2021-06-01 04:49:26 [DEBUG]: Drop duplicates with the same{epitope, CDR3b}
2021-06-01 04:49:26 [DEBUG]: Current df.shape: (11116, 37)


,epitope,epitope_gene,epitope_species,species,cdr3b,mhc,source,label
IKAVYNFATCG_CASSDAGANTEVF,IKAVYNFATCG,None,Lymphocytic choriomeningitis virus (LCMV),Mouse,CASSDAGANTEVF,H-2db,McPAS,1
IKAVYNFATCG_CASSDAGAYAEQF,IKAVYNFATCG,None,Lymphocytic choriomeningitis virus (LCMV),Mouse,CASSDAGAYAEQF,H-2db,McPAS,1
IKAVYNFATCG_CASSDAGGAAEVF,IKAVYNFATCG,None,Lymphocytic choriomeningitis virus (LCMV),Mouse,CASSDAGGAAEVF,H-2db,McPAS,1
IKAVYNFATCG_CASSDAGHSPLYF,IKAVYNFATCG,None,Lymphocytic choriomeningitis virus (LCMV),Mouse,CASSDAGHSPLYF,H-2db,McPAS,1
IKAVYNFATCG_CASSDAWGGAEQYF,IKAVYNFATCG,None,Lymphocytic choriomeningitis virus (LCMV),Mouse,CASSDAWGGAEQYF,H-2db,McPAS,1


(11116, 8)

1: Epitope: AARAVFLAL(9), speices: Melanoma, n_cdr3b: 2
2: Epitope: ACASQKRPSQR(11), speices: Experimental autoimmune encephalomyelitis (EAE), n_cdr3b: 21
3: Epitope: ALIHHNTHL(9), speices: Neoantigen, n_cdr3b: 4
4: Epitope: ALIHHNTYL(9), speices: Neoantigen, n_cdr3b: 2
5: Epitope: ALLETLSLLL(10), speices: Neoantigen, n_cdr3b: 1
6: Epitope: ALLQVTLLL(9), speices: Neoantigen, n_cdr3b: 1
7: Epitope: ALSPVIPHI(9), speices: Neoantigen, n_cdr3b: 16
8: Epitope: ALSPVIPLI(9), speices: Neoantigen, n_cdr3b: 8
9: Epitope: ALSYTPAEV(9), speices: Neoantigen, n_cdr3b: 3
10: Epitope: ALTPVVVTL(9), speices: Acute myeloid leukemia, n_cdr3b: 6
11: Epitope: ALVGAIPSI(9), speices: Neoantigen, n_cdr3b: 1
12: Epitope: ALWGPDPAA(9), speices: Neoantigen, n_cdr3b: 1
13: Epitope: ALYGFVPVL(9), speices: Neoantigen, n_cdr3b: 24
14: Epitope: ALYGSVPVL(9), speices: Diabetes type 2, n_cdr3b: 7
15: Epitope: AMAGSLVFL(9), speices: Neoantigen, n_cdr3b: 3
16: Epitope: AMAGSPVFL(9), speices: Neoantigen, n_cdr3b: 1
17: E

#### SARS-CoV-2 epitope-specific TCRs from Shomuradova et al

- The SARS-CoV-2 epitope S-protein$_{269-277}$(YLQPRTFLL) with the 352 epitope-specific TCRβs from the recent study of Shomuradova et al[{Shomuradova:2020io}].

In [16]:
fn_shomuradova = '%s/Shomuradova/sars2_tcr.tsv' %datadir

##############################
df = pd.read_csv(fn_shomuradova, sep='\t')
df = df[df['Gene'] == 'TRB']
df[CN.epitope] = df['Epitope'].str.strip().str.upper()
df[CN.epitope_gene] = df['Epitope gene']
df[CN.epitope_species] = df['Epitope species']
df[CN.mhc] = df['MHC A']
df[CN.cdr3b] = df['CDR3'].str.strip().str.upper()
df[CN.species] = df['Species']
df[CN.source] = 'Shomuradova'
df[CN.label] = 1

df.index = df.apply(lambda row: get_index(row), axis=1)
df = df.loc[:, CN.values()]

display(df.head(), df.shape, df[CN.epitope].value_counts())

df_shomuradova = df

,epitope,epitope_gene,epitope_species,species,cdr3b,mhc,source,label
YLQPRTFLL_CASSFQNTGELFF,YLQPRTFLL,Spike,SARS-CoV-2,HomoSapiens,CASSFQNTGELFF,HLA-A*02,Shomuradova,1
YLQPRTFLL_CASSSVNNNEQFF,YLQPRTFLL,Spike,SARS-CoV-2,HomoSapiens,CASSSVNNNEQFF,HLA-A*02,Shomuradova,1
YLQPRTFLL_CAVGEANTGELFF,YLQPRTFLL,Spike,SARS-CoV-2,HomoSapiens,CAVGEANTGELFF,HLA-A*02,Shomuradova,1
YLQPRTFLL_CAYQEVNTGELFF,YLQPRTFLL,Spike,SARS-CoV-2,HomoSapiens,CAYQEVNTGELFF,HLA-A*02,Shomuradova,1
YLQPRTFLL_CSARDDQAVNTGELFF,YLQPRTFLL,Spike,SARS-CoV-2,HomoSapiens,CSARDDQAVNTGELFF,HLA-A*02,Shomuradova,1


(381, 8)

YLQPRTFLL    352
RLQSLQTYV     27
VLNDILSRL      1
FIAGLIAIV      1
Name: epitope, dtype: int64

#### Merge positive datasets

-  <strike>After removing duplicates with the same {epitope, CDR3} and selecting all epitope that have at least 30 TCRB CDR3 sequences, the positive dataset contains 12,534 combinations of CDR3b sequences and epitopes with unique 63 epitopes. Table 1 summarizes the positive dataset.</strike>

- After removing duplicates with the same {epitope, CDR3}, the positive dataset contains 14,352 combinations of CDR3b sequences and epitopes with unique 491 epitopes. Table 1 summarizes the positive dataset.

In [19]:
import pandas as pd

n_cdr3b_cutoff = 20
###################################################

logger.debug('Merge all datasets')
df = pd.concat([df_dash, df_vdjdb, df_mcpas, df_shomuradova])
logger.debug('Current df.shape: %s' % str(df.shape))

logger.debug('Drop duplicates')
df = df[~df.index.duplicated()]
logger.debug('Current df.shape: %s' % str(df.shape))

# logger.debug('Select all epitope with at least 30 TCRB sequences')
# tmp = df[CN.epitope].value_counts()
# tmp = tmp[tmp >= n_cdr3b_cutoff]
# df = df[df[CN.epitope].map(lambda x: x in tmp.index)]
# logger.debug('Final df.shape: %s' % str(df.shape))

display(df.head(), df.shape)

for i, (epitope, subdf) in enumerate(df.groupby([CN.epitope])):
    epitope_gene = subdf[CN.epitope_gene].unique()
    source = subdf[CN.source].unique()
    epitope_species = subdf[CN.epitope_species].iat[0]
    n_cdr3b = subdf.shape[0]
    print('%s: Epitope: %s(%s), speices: %s, gene: %s, n_cdr3b: %s, source: %s' % (i + 1, epitope, len(epitope), 
                                                                       epitope_species, epitope_gene, n_cdr3b,
                                                                       source))

df_train_pos = df

2021-06-01 04:02:59 [DEBUG]: Merge all datasets
2021-06-01 04:02:59 [DEBUG]: Current df.shape: (17160, 8)
2021-06-01 04:02:59 [DEBUG]: Drop duplicates
2021-06-01 04:02:59 [DEBUG]: Current df.shape: (14352, 8)


,epitope,epitope_gene,epitope_species,species,cdr3b,mhc,source,label
GLCTLVAML_CASSLWTGSHEQYF,GLCTLVAML,BMLF,EBV,human,CASSLWTGSHEQYF,HLA-A*02:01,Dash,1
GLCTLVAML_CASSQSPGGEQYF,GLCTLVAML,BMLF,EBV,human,CASSQSPGGEQYF,HLA-A*02:01,Dash,1
GLCTLVAML_CASSLTTEQQFF,GLCTLVAML,BMLF,EBV,human,CASSLTTEQQFF,HLA-A*02:01,Dash,1
GLCTLVAML_CSARDATGNGYTF,GLCTLVAML,BMLF,EBV,human,CSARDATGNGYTF,HLA-A*02:01,Dash,1
GLCTLVAML_CSARDSTGNGYTF,GLCTLVAML,BMLF,EBV,human,CSARDSTGNGYTF,HLA-A*02:01,Dash,1


(14352, 8)

1: Epitope: AAFKRSCLK(9), speices: MCPyV, gene: ['T-Ag'], n_cdr3b: 5, source: ['VDJdb']
2: Epitope: AAGIGILTV(9), speices: HomoSapiens, gene: ['MLANA'], n_cdr3b: 2, source: ['VDJdb']
3: Epitope: AARAVFLAL(9), speices: Melanoma, gene: [None], n_cdr3b: 2, source: ['McPAS']
4: Epitope: ACASQKRPSQR(11), speices: Experimental autoimmune encephalomyelitis (EAE), gene: [None], n_cdr3b: 21, source: ['McPAS']
5: Epitope: ALDPHSGHFV(10), speices: HomoSapiens, gene: ['CDK4'], n_cdr3b: 4, source: ['VDJdb']
6: Epitope: ALGIGILTV(9), speices: HomoSapiens, gene: ['MLANA'], n_cdr3b: 1, source: ['VDJdb']
7: Epitope: ALIHHNTHL(9), speices: Neoantigen, gene: [None], n_cdr3b: 4, source: ['McPAS']
8: Epitope: ALIHHNTYL(9), speices: Neoantigen, gene: [None], n_cdr3b: 2, source: ['McPAS']
9: Epitope: ALLETLSLLL(10), speices: Neoantigen, gene: [None], n_cdr3b: 1, source: ['McPAS']
10: Epitope: ALLQVTLLL(9), speices: Neoantigen, gene: [None], n_cdr3b: 1, source: ['McPAS']
11: Epitope: ALSPVIPHI(9), speices: Ho

179: Epitope: IVTDFSVIK(9), speices: EBV, gene: ['EBNA4' None], n_cdr3b: 36, source: ['VDJdb' 'McPAS']
180: Epitope: KAFSPEVIPMF(11), speices: HIV-1, gene: ['Gag' None], n_cdr3b: 172, source: ['VDJdb' 'McPAS']
181: Epitope: KASEKIFYV(9), speices: HomoSapiens, gene: ['SSX2'], n_cdr3b: 4, source: ['VDJdb']
182: Epitope: KGYVYQGL(8), speices: VSV, gene: ['N'], n_cdr3b: 9, source: ['VDJdb']
183: Epitope: KIFGSLAFL(9), speices: HomoSapiens, gene: ['ERBB2'], n_cdr3b: 1, source: ['VDJdb']
184: Epitope: KINAWIKVV(9), speices: HIV-1, gene: ['Gag'], n_cdr3b: 3, source: ['VDJdb']
185: Epitope: KIVALGINAV(10), speices: Neoantigen, gene: [None], n_cdr3b: 4, source: ['McPAS']
186: Epitope: KLAKPLPYT(9), speices: Neoantigen, gene: [None], n_cdr3b: 1, source: ['McPAS']
187: Epitope: KLANPLPYT(9), speices: Neoantigen, gene: [None], n_cdr3b: 2, source: ['McPAS']
188: Epitope: KLEFMDWRL(9), speices: synthetic, gene: ['synthetic'], n_cdr3b: 1, source: ['VDJdb']
189: Epitope: KLEQLDWTV(9), speices: HomoSap

489: Epitope: YTSEILSPI(9), speices: Neoantigen, gene: [None], n_cdr3b: 1, source: ['McPAS']
490: Epitope: YVLDHLIVV(9), speices: EBV, gene: ['BRLF1' None], n_cdr3b: 92, source: ['VDJdb' 'McPAS']
491: Epitope: YVLTWIVGA(9), speices: Neoantigen, gene: [None], n_cdr3b: 6, source: ['McPAS']


In [20]:
14352 * 2

28704

### Negative dataset

- To increase the specificity of the predictive model, negative examples wich we do not expect to interact between TCRs and epitopes were added.
- The negative examples were made by combining the peptides from the positive dataset with the randomly selected background TCR CDR3 beta sequences constructed by Dash et al[{Dash:2017go}] which have been collected from two healty donors[{Howie:2015dc}]
- 예측 모델의 학습에서의 Overfitting을 피하기 위해, trainging 데이터셋은 epitope 별로 같은 수의 positive/negative data point를 포함하도록 하였다. 최종 Training 데이터셋은 28,704개의 TCR CDR3b-peptide combinations를 포함하고 있다(Table S1)

In [21]:
fn_cntr = '%s/TCRGP/human_tcr_control.csv' % datadir
################################
df_cntr = pd.read_csv(fn_cntr)
display(df_cntr.head())
pos_cdr3b = df_train_pos[CN.cdr3b].unique()
neg_cdr3b = list(filter(lambda x: x not in pos_cdr3b, df_cntr['cdr3b'].unique()))

print('len(pos_cdr3b): %s, len(neg_cdr3b): %s' % (len(pos_cdr3b), len(neg_cdr3b)))

df = pd.DataFrame(columns=CN.values())
for epitope, subdf in df_train_pos.groupby([CN.epitope]):
    subdf_neg = subdf.copy()
    subdf_neg[CN.source] = 'Control'
    subdf_neg[CN.label] = 0
    subdf_neg[CN.cdr3b] = np.random.choice(neg_cdr3b, subdf.shape[0], replace=False)
    subdf_neg.index = subdf_neg.apply(lambda row: get_index(row), axis=1)
    df = df.append(subdf_neg)

display(df.head(), df.shape)    
df_train_neg = df

,va_reps,vb_reps,cdr3a,cdr3b
0,TRAV9-2*01,TRBV7-9*01,CALSPGGTSYGKLTF,CASSPATGGLVDTQYF
1,TRAV9-2*01,TRBV20/OR9-2*01;TRBV20-1*01,CALKGYTGRRALTF,CSAREGLAGDYEQYF
2,TRAV9-2*01,TRBV5-1*01,CALSTGANSKLTF,CASRRALAGADEQYF
3,TRAV6*01,TRBV27*01,CALDMRSGAGSYQLTF,CASRPFQVSTDTQYF
4,TRAV29/DV5*01,TRBV7-6*01,CAAPFKGGSEKLVF,CASSYSPHNSPLHF


len(pos_cdr3b): 13715, len(neg_cdr3b): 8039


,epitope,epitope_gene,epitope_species,species,cdr3b,mhc,source,label
AAFKRSCLK_CASSSGRFERAAHDTQYF,AAFKRSCLK,T-Ag,MCPyV,HomoSapiens,CASSSGRFERAAHDTQYF,HLA-A*03:01,Control,0
AAFKRSCLK_CASSLAGPSSAQHF,AAFKRSCLK,T-Ag,MCPyV,HomoSapiens,CASSLAGPSSAQHF,HLA-A*03:01,Control,0
AAFKRSCLK_CASSFGLAGGPNTGELFF,AAFKRSCLK,T-Ag,MCPyV,HomoSapiens,CASSFGLAGGPNTGELFF,HLA-A*03:01,Control,0
AAFKRSCLK_CASSSTGVSYEQYF,AAFKRSCLK,T-Ag,MCPyV,HomoSapiens,CASSSTGVSYEQYF,HLA-A*03:01,Control,0
AAFKRSCLK_CASRTGLGEQYF,AAFKRSCLK,T-Ag,MCPyV,HomoSapiens,CASRTGLGEQYF,HLA-A*03:01,Control,0


(14352, 8)

In [22]:
fn_train = '%s/train.csv' % outdir

df_train = pd.concat([df_train_pos, df_train_neg])

display(df_train.head(), df_train.shape)
print('Duplicated count: %s' % np.count_nonzero(df_train.index.duplicated()))
max_len = df_train.index.map(lambda x: len(x) - 1).max()
print('max_len: %s' % max_len)
df_train.to_csv(fn_train)

,epitope,epitope_gene,epitope_species,species,cdr3b,mhc,source,label
GLCTLVAML_CASSLWTGSHEQYF,GLCTLVAML,BMLF,EBV,human,CASSLWTGSHEQYF,HLA-A*02:01,Dash,1
GLCTLVAML_CASSQSPGGEQYF,GLCTLVAML,BMLF,EBV,human,CASSQSPGGEQYF,HLA-A*02:01,Dash,1
GLCTLVAML_CASSLTTEQQFF,GLCTLVAML,BMLF,EBV,human,CASSLTTEQQFF,HLA-A*02:01,Dash,1
GLCTLVAML_CSARDATGNGYTF,GLCTLVAML,BMLF,EBV,human,CSARDATGNGYTF,HLA-A*02:01,Dash,1
GLCTLVAML_CSARDSTGNGYTF,GLCTLVAML,BMLF,EBV,human,CSARDSTGNGYTF,HLA-A*02:01,Dash,1


(28704, 8)

Duplicated count: 0
max_len: 42


### Independent external dataset

#### SARS-Cov-2 epitopes from ImmuneCODE

- The SARS-CoV-2 epitope S-protein$_{269-277}$(YLQPRTFLL) with the 451 epitope-specific TCRβs from the ImmuneRACE study launched by Adaptive Biotechnologies and Microsoft (https://immunerace.adaptivebiotech.com, June 10, 2020 dataset)

In [23]:
fn_ic = '%s/ImmuneCODE/sars2_YLQPRTFLL_with_neg.csv' % datadir


df = pd.read_csv(fn_ic)

df[CN.epitope] = 'YLQPRTFLL'
df[CN.epitope_gene] = 'Spike'
df[CN.epitope_species] = 'SARS-CoV-2'
df[CN.mhc] = None
df[CN.cdr3b] = df['cdr3b'].str.strip().str.upper()
df[CN.species] = 'human'
df[CN.source] = 'ImmuneCODE'
df[CN.label] = df['subject'].map(lambda x: 0 if x == 'control' else 1)

df.index = df.apply(lambda row: get_index(row), axis=1)
df = df.loc[:, CN.values()]

display(df.head(), df.shape, df.label.value_counts())

df_immunecode = df

,epitope,epitope_gene,epitope_species,species,cdr3b,mhc,source,label
YLQPRTFLL_CAKGLANTGELFF,YLQPRTFLL,Spike,SARS-CoV-2,human,CAKGLANTGELFF,None,ImmuneCODE,1
YLQPRTFLL_RKQWQGEETQYF,YLQPRTFLL,Spike,SARS-CoV-2,human,RKQWQGEETQYF,None,ImmuneCODE,1
YLQPRTFLL_CASSYRNTGELFF,YLQPRTFLL,Spike,SARS-CoV-2,human,CASSYRNTGELFF,None,ImmuneCODE,1
YLQPRTFLL_CSARDNWAVNTGELFF,YLQPRTFLL,Spike,SARS-CoV-2,human,CSARDNWAVNTGELFF,None,ImmuneCODE,1
YLQPRTFLL_CASSPDIEAFF,YLQPRTFLL,Spike,SARS-CoV-2,human,CASSPDIEAFF,None,ImmuneCODE,1


(830, 8)

0    415
1    415
Name: label, dtype: int64

## PyTorch Dataset Class

In [ ]:
!pip install tape_proteins

import tape

tape.__version__

In [24]:
import torch
from torch.utils.data import Dataset
import numpy as np
from tape import TAPETokenizer

class TCREpitopeSentenceDataset(Dataset):
    
    def __init__(self, df=None, max_len=None):
        self._df = df
        self.max_len = max_len

    def __getitem__(self, index):
        
        row = self._df.iloc[index, :]
        sequence_ids = eval(row['sequence'])
        label = row[CN.label]
        
        return torch.tensor(sequence_ids), torch.tensor(label)

    def __len__(self):
        return self._df.shape[0]
    
    @classmethod
    def from_data_file(cls, fn=None, max_len=None, tokenizer=None):
        def encode_row(row):
            epitope = row[CN.epitope]
            cdr3b = row[CN.cdr3b]
            logger.debug('Encoding epitope: %s, cdr3b: %s' % (epitope, cdr3b))
            sequence_ids = tokenizer.encode(epitope)
            
            sequence_ids = np.append(sequence_ids, tokenizer.encode(cdr3b))
            n_pads = max_len - sequence_ids.shape[0]
            if n_pads > 0:
                sequence_ids = np.append(sequence_ids, [tokenizer.vocab['<pad>']]*n_pads)
            return sequence_ids
            
        df = pd.read_csv(fn)
        df['sequence'] = df.apply(encode_row, axis=1)
        
        return cls(df=df, max_len=max_len)
        
        
##########################################
tokenizer = TAPETokenizer(vocab='iupac')
train_ds = TCREpitopeSentenceDataset.from_data_file(fn=fn_train, max_len=max_len, tokenizer=tokenizer)

2021-06-01 04:06:01 [INFO]: Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GLCTLVAML, cdr3b: CASSLWTGSHEQYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GLCTLVAML, cdr3b: CASSQSPGGEQYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GLCTLVAML, cdr3b: CASSLTTEQQFF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GLCTLVAML, cdr3b: CSARDATGNGYTF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GLCTLVAML, cdr3b: CSARDSTGNGYTF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GLCTLVAML, cdr3b: CSARDRTGNGYTF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GLCTLVAML, cdr3b: CSARGQGDEQYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GLCTLVAML, cdr3b: CASSSGAGLDEQFF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GLCTLVAML, cdr3b: CASSQSPGGTQYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GLCTLVAML, cdr3b: CASGGSSYQETQYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GLCTLVAML, cdr3b: CSVGSAGTNEKLFF
2021-06-01

2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GILGFVFTL, cdr3b: CASSGGRNQPQHF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GILGFVFTL, cdr3b: CASSPLIGEQYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GILGFVFTL, cdr3b: CSARVRGDVYGYTF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GILGFVFTL, cdr3b: CASSQVEAFYEQYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GILGFVFTL, cdr3b: CASSLERNTYEQYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GILGFVFTL, cdr3b: CASSQDPGYEQYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GILGFVFTL, cdr3b: CASSYSGHVNTEAFF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GILGFVFTL, cdr3b: CSARDGTRAHYEQFF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GILGFVFTL, cdr3b: CASSIFGGAEAFF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GILGFVFTL, cdr3b: CASSQDWGAGTQYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: NLVPMVATV, cdr3b: CATEQGMHEQYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: NLVPMVATV, cdr3b: CASSPITGTGAYGYTF
2021-06-01 04:06:01 [DEBUG]: Encoding epito

2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GILGFVFTL, cdr3b: CASSIRSGVEQYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GILGFVFTL, cdr3b: CASSILTGPRTEAFF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GILGFVFTL, cdr3b: CASSFRSTDTQYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GILGFVFTL, cdr3b: CASSSRSANEQFF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GILGFVFTL, cdr3b: CASSPRASLEQYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GILGFVFTL, cdr3b: CASSIAPTDTQYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: NLVPMVATV, cdr3b: CAVNGGSNYGYTF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: NLVPMVATV, cdr3b: CAWSVSDVANTEAFF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: NLVPMVATV, cdr3b: CASSYFDEKLFF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GLCTLVAML, cdr3b: CASSQSPGGQQFF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GLCTLVAML, cdr3b: CSVGSDGTNEKLFF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: GLCTLVAML, cdr3b: CSARVEPGNGYTF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: G

2021-06-01 04:06:01 [DEBUG]: Encoding epitope: SSPPMFRV, cdr3b: CASGGNYAEQFF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: SSPPMFRV, cdr3b: CASSPGTGGYNYAEQFF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: SSPPMFRV, cdr3b: CATGTGSSAETLYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: SSPPMFRV, cdr3b: CASSGGTGGFNYAEQFF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: SSPPMFRV, cdr3b: CTCSDGTGGYNYAEQFF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: SSPPMFRV, cdr3b: CTCSPGTGGVNYAEQFF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: SSYRRPVGI, cdr3b: CSSSQSSAETLYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: SSYRRPVGI, cdr3b: CASSLPDSQNTLYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: SSYRRPVGI, cdr3b: CASSGQSNERLFF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: SSYRRPVGI, cdr3b: CASSFPDSAETLYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: SSYRRPVGI, cdr3b: CASSQDSAETLYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: SSYRRPVGI, cdr3b: CASKGRTSAETLYF
2021-06-01 04:06:01 [DEBUG]: Encoding e

2021-06-01 04:06:01 [DEBUG]: Encoding epitope: ASNENMETM, cdr3b: CASSLTAISNERLFF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: ASNENMETM, cdr3b: CASSPWGGADQNTLYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: ASNENMETM, cdr3b: CASGEDWGKDTQYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: ASNENMETM, cdr3b: CASSRDWGDAEQFF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: ASNENMETM, cdr3b: CASSQDLGGRYEQYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: ASNENMETM, cdr3b: CASSQGLGGRGEQYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: ASNENMETM, cdr3b: CLSIQDRRVSYNSPLYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: SSYRRPVGI, cdr3b: CASSGDSSAETLYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: SSYRRPVGI, cdr3b: CASSYSSAETLYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: SSYRRPVGI, cdr3b: CASSSDSAETLYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: SSYRRPVGI, cdr3b: CASSLDSAETLYF
2021-06-01 04:06:01 [DEBUG]: Encoding epitope: ASNENMETM, cdr3b: CASSQEDWGSAETLYF
2021-06-01 04:06:01 [DEBUG]: Enco

...

2021-06-01 04:06:04 [DEBUG]: Encoding epitope: MEVGWYRPPFSRVVHLYRNGK, cdr3b: CASGETGGVYAEQF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: ELAGIGILTV, cdr3b: CASRLATLVGNEQF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: ELAGIGILTV, cdr3b: CASRSGSLNTEAF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: ELAGIGILTV, cdr3b: CASRWGQLTNYGYF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: ELAGIGILTV, cdr3b: CASSENQLNMTEAF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: ELAGIGILTV, cdr3b: CASSIAAVSALYGYTLF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: ELAGIGILTV, cdr3b: CASSKPSGGINEQF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: ELAGIGILTV, cdr3b: CASSLQGLGTEAF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: ELAGIGILTV, cdr3b: CASSLTTPPSTDPQYF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: ELAGIGILTV, cdr3b: CASSPGQGLPNHF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: ELAGIGILTV, cdr3b: CASSQGQLSSYNEQF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: ELAGIGILTV, cdr3b: CASSQQGLGTEAF
2021-06-01 04:0

2021-06-01 04:06:04 [DEBUG]: Encoding epitope: NLVPMVATV, cdr3b: CASTSWAQASTDTQYF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: NLVPMVATV, cdr3b: CASTVDSLDTEAF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: NLVPMVATV, cdr3b: CATSDLGQGLDNEQF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: NLVPMVATV, cdr3b: CSAPDLFGQGIETQYF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: NLVPMVATV, cdr3b: CSAQSGLAGGSLQETQYF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: NLVPMVATV, cdr3b: CSARDAGLAGLNGAPSSYEQYF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: NLVPMVATV, cdr3b: CSARDPFGTGYLNSNQPQHF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: NLVPMVATV, cdr3b: CSAREPDDTREAF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: NLVPMVATV, cdr3b: CSARGDTGTGYEQYF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: NLVPMVATV, cdr3b: CSARHDLGQGYEQYF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: NLVPMVATV, cdr3b: CSARTTYGTDIISQHF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: NLVPMVATV, cdr3b: CSARVVPGVQETQYF
2021-06-01 04:

2021-06-01 04:06:04 [DEBUG]: Encoding epitope: GLCTLVAML, cdr3b: CSVPGTDYNEQF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: GLCTLVAML, cdr3b: CSVPSLLNTEAF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: GLCTLVAML, cdr3b: CSVQEGGYGYTF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: RAKFKQLL, cdr3b: CSVTAGEDDTQYF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: RAKFKQLL, cdr3b: CSVTAGEDQPQHF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: RAKFKQLL, cdr3b: CSVTSERTGTTWGYTF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: RAKFKQLL, cdr3b: CSYSPGQGRNEQF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: TQGYFPDWQNY, cdr3b: CAISECAIRGTSGLTDTQYF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: TQGYFPDWQNY, cdr3b: CAISERAIRGTSGLTDTQYL
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: EIYKRWII, cdr3b: CASELSGNTIYF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: FLKEKGGL, cdr3b: CASKEELSNTGELF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: FLKEKGGL, cdr3b: CASRGGSGELF
2021-06-01 04:06:04 [DEBUG]: Encoding epito

2021-06-01 04:06:04 [DEBUG]: Encoding epitope: KRWIILGLNK, cdr3b: CASSQGQLGNTIYF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: KRWIILGLNK, cdr3b: CASSQGQYGNEQF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: KRWIILGLNK, cdr3b: CASSQPPDRGYGYTF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: KRWIILGLNK, cdr3b: CASSRTQGPNTDTQYF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: KRWIILGLNK, cdr3b: CASSSKTGELF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: KRWIILGLNK, cdr3b: CASSSRTGELF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: KRWIILGLNK, cdr3b: CASSSTRGTEAF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: KRWIILGLNK, cdr3b: CASSTTSGRYEQYF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: KRWIILGLNK, cdr3b: CASSVLGTSGGAEQF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: KRWIILGLNK, cdr3b: CATSETGELF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: KRWIILGLNK, cdr3b: CATSGVTGELF
2021-06-01 04:06:04 [DEBUG]: Encoding epitope: KRWIILGLNK, cdr3b: CAWSLTGMNQPQHF
2021-06-01 04:06:04 [DEBUG]: Encoding ep

KeyError: "Unrecognized token: '/'"